In [19]:
from random import choice
from enbios2.base.experiment import Experiment
import bw2data

from enbios2.bw2.util import report
from enbios2.models.experiment_models import ExperimentData

In [20]:
# get an overview of brighway projects and databases
report()

Project: default
['biosphere3']
Project: ecoinvent
['biosphere3', 'cutoff_3.9.1_default']


In [21]:
bw2data.projects.set_current("ecoinvent")
database_name = 'cutoff_3.9.1_default'
db = bw2data.Database(database_name)

# Simple example experiment
In this first simple example we calculate the impact of 4 wind turbine activities in spain, using 2 different methods.

In [22]:
wind_turbines_spain = db.search("electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"})
wind_turbines_spain

Excluding 319 filtered results


['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, ES, None),
 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, ES, None)]

In [23]:
# for the experiment we need to create a list of activities (or a dict, where the keys represent the aliases)
# We need to add the codes, otherwise the brightway search will not be not uniquely identify the activities 
# adding name is just for convenience
experiment_activities = []

for activity in wind_turbines_spain:
    experiment_activities.append(
        {"id":
            {
                "name": activity["name"],
                "code": activity["code"]
            }
        }
    )

In [24]:
# we can modify the output of the activities, by default it is the reference product (1 of the activity unit)
experiment_activities[0]["output"] = ["kilowatt_hour", 3]
experiment_activities

[{'id': {'name': 'electricity production, wind, >3MW turbine, onshore',
   'code': '0d48975a3766c13e68cedeb6c24f6f74'},
  'output': ['kilowatt_hour', 3]},
 {'id': {'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'code': 'ed3da88fc23311ee183e9ffd376de89b'}},
 {'id': {'name': 'electricity production, wind, 1-3MW turbine, offshore',
   'code': '6ebfe52dc3ef5b4d35bb603b03559023'}},
 {'id': {'name': 'electricity production, wind, <1MW turbine, onshore',
   'code': '72cc067e1f4093c2e4c6ac9bdc93d844'}}]

In [25]:
# select 2 random methods and convert them into the form for enbios2
all_methods = list(bw2data.methods)
methods = [choice(all_methods) for _ in range(2)]
experiment_methods = [
    {
        "id": method
    }
    for method in methods
]

experiment_methods

[{'id': ('EDIP 2003', 'non-renewable resources', 'copper')},
 {'id': ('Cumulative Exergy Demand (CExD)',
   'energy resources: non-renewable, fossil',
   'exergy content')}]

In [26]:
# make a first validation of the experiment data
exp_data = ExperimentData(
    bw_project="ecoinvent",
    activities=experiment_activities,
    methods=experiment_methods
)

In [27]:
# create a experiment object. This will validate the activities, their outputs, the methods and the scenarios.
exp: Experiment = Experiment(exp_data)

In [28]:
print(exp.info())

Experiment: 
Activities: 4
  electricity production, wind, >3MW turbine, onshore - electricity production, wind, >3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, onshore - electricity production, wind, 1-3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, offshore - electricity production, wind, 1-3MW turbine, offshore
  electricity production, wind, <1MW turbine, onshore - electricity production, wind, <1MW turbine, onshore
Methods: 2
 ('EDIP 2003', 'non-renewable resources', 'copper')
 ('Cumulative Exergy Demand (CExD)', 'energy resources: non-renewable, fossil', 'exergy content')
Scenarios: 1


In [29]:
# run all scenarios at once
results = exp.run()

2023-08-02 17:06:24,733 - bw2calc - INFO - Initialized LCA object. Demand: {25415: 1, 15150: 1, 25751: 1, 25926: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x7f433dda4f10>, <bw_processing.datapackage.Datapackage object at 0x7f433e591f90>, <bw_processing.datapackage.Datapackage object at 0x7f433e8e5110>, <bw_processing.datapackage.Datapackage object at 0x7f433e5d6d10>, <bw_processing.datapackage.Datapackage object at 0x7f433e5e7750>, <bw_processing.datapackage.Datapackage object at 0x7f433dda6b50>]
2023-08-02 17:06:24,734 - bw2calc - INFO - {'message': 'Started MultiLCA calculation', 'methods': [('EDIP 2003', 'non-renewable resources', 'copper'), ('Cumulative Exergy Demand (CExD)', 'energy resources: non-renewable, fossil', 'exergy content')], 'functional units': [[{'database': 'cutoff_3.9.1_default', 'code': '0d48975a3766c13e68cedeb6c24f6f74', 'amount': 3.0}], [{'database': 'cutoff_3.9.1_default', 'code': 'ed3da88fc23311ee183e9ffd376de89b', 'amount': 1.0}], [{'dat

# Result
The result is a dictionary of scenario names, where for each scenario we have a tree (representing the activity hierarchy). Each node (`BasicTreeNode`) in the tree has a `data` object, which is of the type `ScenarioResultNodeData`, which have the fields `output`, `result` and `bw_activity`.

In [30]:
# from enbios2.generic.tree.basic_tree import BasicTreeNode
# from enbios2.models.experiment_models import ScenarioResultNodeData

results

{'default scenario': [root - 4 children ]}

In [31]:
print(results["default scenario"].info())
print("---")
for children in results["default scenario"]:
    print(children.info())
    print("---")

[root - 4 children ]
ScenarioResultNodeData(output=('kilowatt_hour', 6.0), results={'EDIP 2003_non-renewable resources_copper': 0.0009922747918774807, 'Cumulative Exergy Demand (CExD)_energy resources: non-renewable, fossil_exergy content': 1.3475895192549352}, bw_activity=None)
---
[electricity production, wind, >3MW turbine, onshore - 0 children  (root)]
ScenarioResultNodeData(output=('kilowatt_hour', 3.0), results={'EDIP 2003_non-renewable resources_copper': 0.0008615528096952267, 'Cumulative Exergy Demand (CExD)_energy resources: non-renewable, fossil_exergy content': 0.8146299248462172}, bw_activity='electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None))
---
[electricity production, wind, 1-3MW turbine, onshore - 0 children  (root)]
ScenarioResultNodeData(output=('kilowatt_hour', 1.0), results={'EDIP 2003_non-renewable resources_copper': 3.698917801892879e-05, 'Cumulative Exergy Demand (CExD)_energy resources: non-renewable, fossil_exergy content': 0.18504

In [18]:
# we can dump the results into a csv file
exp.results_to_csv("demo_result.csv")
# exp.scenarios[0].results_to_csv("test.csv")

In [ ]:
exp.scenarios[0].result_to_dict()